<a href="https://colab.research.google.com/github/RedBatProject/Thesis---a-two-layer-power-network/blob/main/Doing_thesis_sep1th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All the imports here

In [1]:
%pip install pandapower
import pandapower as pp
import networkx as nx
import pandapower.networks as pn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandapower.topology as top
# import networkx as nx
from pandapower import plotting
from pandapower.plotting import simple_plot, pf_res_plotly,simple_plotly
import random as rn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numba
# plt.rcParams['figure.dpi'] = 100
# plt.rcParams['savefig.dpi'] = 100

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 7.6 MB/s eta 0:00:00
  Created wheel for pandapower: filename=pandapower-2.13.1-py3-none-any.whl size=10803675 sha256=f3d7ebe756cc5f00f1c1fd9a208bff9dc3da56d46b3d55ae3c8b62ece091dff8
  Stored in directory: /root/.cache/pip/wheels/6b/58/fb/c5c16044f0046e905e2fe55de3af6c0a43bfcc3219ed5d3af8
Successfully built pandapower


In [12]:
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
seed = 1537
alpha = 0.39
beta = 0.1
gamma = 1-(alpha + beta)
alpha,beta,gamma
np.random.seed(seed)


nodes=101
k=4
# seed=seed
# np.random.seed(seed)

p=0.2
types=0
# G = nx.watts_strogatz_graph(nodes,k,p=p,seed=seed)
# edges = list(G.edges())

# The Code

In [5]:
def remove_by_fraction(numer_of_nodes,of_set):
    p_ = list()
    f_ = int(numer_of_nodes * of_set)
    while len(p_) < f_:
        f1_ = np.random.randint(0,numer_of_nodes)
        if f1_ not in p_ and f1_ != target:
            p_.append(f1_)
    return p_

def remove_by_number(numer_of_nodes,of_set):
    p_ = list()
    f_ = int(of_set)
    while len(p_) < f_:
        f1_ = np.random.randint(0,numer_of_nodes)
        if f1_ not in p_ and f1_ != target:
            p_.append(f1_)
    return p_

# define set of nodes which you want to remove
def set_of_nodes(numer_of_nodes,of_set):
    if of_set < 1:
        p_frac = remove_by_fraction(numer_of_nodes,of_set)
    if of_set >= 1:
        p_frac = remove_by_number(numer_of_nodes,of_set)
    return np.array(p_frac)


In [6]:
# Ok, we have a Net, no we want to create the communication network, for this we use our written model from before
def create_net(net):
    # np.random.seed(seed)

    Net = net
    G_power = top.create_nxgraph(net=Net)
    # extract some feature for future
    G_power_nodes = G_power.nodes()
    G_power_nodes_Number = len(G_power_nodes)
    G_power_edges = G_power.edges()
    target = int(Net.ext_grid['bus'][0])
    Static_node_of_power = len(Net.bus)
    G_scale_free = nx.scale_free_graph(n= Static_node_of_power, alpha= alpha, beta= beta, gamma= gamma, seed= seed, delta_in=1, delta_out=1 )
        # create Com from Scale free
    # print(set(G_scale_free.nodes()))
    G_com_edges = G_scale_free.edges()
    G_com_edges_set = set(G_com_edges)
    G_com_edges_set_non_redundancy = []
    # print(set(G_com.nodes()))
    for G_edge in G_com_edges_set:
        if (G_edge[1],G_edge[0]) and (G_edge[0],G_edge[1]) not in G_com_edges_set_non_redundancy:
            if G_edge[1] != G_edge[0]:
                G_com_edges_set_non_redundancy.append(G_edge)
    G_com = nx.MultiGraph()
    # print(set(G_com.nodes()))
    G_com.add_edges_from(G_com_edges_set_non_redundancy);
    if len(set(G_scale_free.nodes() - set(G_com.nodes))) > 0:
        for ij in list(set(G_scale_free.nodes() - set(G_com.nodes))):
            G_com.add_edges_from([(ij,np.random.randint(len(G_power.nodes())))])
    # print(set(G_com.nodes()))
        # add a few work to this
    mapping = {0:target,target:0}
    G_com = nx.relabel_nodes(G_com, mapping)

        # extract initial information
    G_com_nodes = G_com.nodes()
    # print(set(G_com.nodes()))
    G_com_edges = G_com.edges()
    # print("Networks settings are Done!")
    return G_com,Net,G_power,target,G_power_nodes_Number

In [9]:
net = pn.case30()
# nodes = 300
G_com,Net,G_power,target,G_power_nodes_Number = create_net(net)

In [10]:
nodes_from_both = {'power':set(G_power.nodes),'communication':set(G_com.nodes),'disabled':set()}
# nodes_from_both

In [11]:
# we sould find the largest component
# we use this code from networkx
def Largest_component_with_target(G,target_node):
    # np.random.seed(seed)

    # check the largest components
    components_of_groph = sorted(nx.connected_components(G), key=len, reverse=True)
    for component in components_of_groph:
        if target_node in component:
            G = G.subgraph(component)
            G = nx.MultiGraph(G)
    return G

In [53]:
# Again we write a code for casecading failure in this time
# here are what we should do
# 1: drop the removed_nodes from power network
# 2: drop the coupled_nodes from communication network
# 3: modify the power network
# 4: let the casecade goes and see where it could go
# 5: evaluate

# initiate the p_frac
def phase_0(p_frac):
    # print('im in phase 0')
    if len(p_frac) == 0:
        # self.node = instance_nodes - nodes
        busses = []
    elif len(p_frac) == 1:
        busses = p_frac
    else:

        busses = list(p_frac)
    return busses


# first time removing nodes
def phase_0_1(Net,busses):
    # print('im in phase 0_1')
    # try:
    pp.drop_buses(Net,buses=busses,drop_elements = True)
    # except:print('[SOMETHING HAPPENED!]')
    return Net
# removing nodes from power network
def phase_1(Net):
    # print('im in phase 1')
    # we run power for the first time
    pp.rundcpp(Net)
    # drop all droped element relevants
    drop_lines = pp.overloaded_lines(Net)
    # drop the lines
    pp.drop_lines(Net,drop_lines)
    # create a nx network from power network
    g_inst_power = top.create_nxgraph(Net)
    # look for largest copunent
    g_inst_2_power = Largest_component_with_target(g_inst_power,target)
    # which nodes are remvoved from network?
    set_nodes_2 = set(g_inst_power) - set(g_inst_2_power)
    pp.drop_buses(Net,set_nodes_2,drop_elements = True)
    return Net,g_inst_2_power,set_nodes_2

# now we know what nodes from communication part should get remove
def phase_2(G_com,to_com):
    # print('im in phase 2')
    G_com.remove_nodes_from(to_com)
    g_inst_2 = Largest_component_with_target(G_com,target)
    set_nodes_2 = set(G_com.nodes) - set(g_inst_2.nodes)
    return g_inst_2,set_nodes_2

# now the modification part
def phase_3(net,gen_frac,pow_frac,is_not_in):
    # print('im in phase 3')
    imp_gen = list(net.gen['p_mw'].sort_values(ascending=False).index)
    if gen_frac <= 1:
        s_number = int(len(imp_gen) * gen_frac)
    else:
        s_number = int(gen_frac)
    # for i in imp_gen[0:s_number]:
    for i in range(0,s_number):
        if i not in is_not_in:
            # print(i)
            # print(imp_gen)
            # print(imp_gen[i])
            net.gen['p_mw'].loc[imp_gen[i]] *= 1 + pow_frac
            # print(f"oh, {i} is modified!")
        else:
            pass
            # print("cant modify {} cause we cant reach it".format(i))
    return net

# now we should continue the casecading failure
def phase_4(Net,G_power,G_com):
    # print('im in phase 4')
    Net,G_power,to_com = phase_1(Net)
    while len(list(to_com)) != 0:
        Net,G_power,to_com = phase_1(Net)
        # print(to_com)
        # G_com,to_power = phase_2(G_com,to_com)
    return Net,G_power,G_com
def casecading_version_2(Net,G_com,G_power,p_frac,fraction_gen,fraction_power):
    nodes_from_both = {'power':set(G_power.nodes),'communication':set(G_com.nodes),'disabled':set()}

    busses = phase_0(p_frac)
    # remove nodes from power
    Net = phase_0_1(Net,busses)
    # remove nodes from power network
    Net,G_power,to_com = phase_1(Net)
    # set the power reminded node
    nodes_from_both['power'] = set(G_power.nodes)
    # we should remove them from communiaction network too
    if len(list(to_com)) != 0:
        # remove nodes from communication network
        G_com,to_power = phase_2(G_com,to_com)
        # set the communication reminded nodes
        nodes_from_both['communication'] = set(G_com.nodes)
        # set the disabled nodes
        nodes_from_both['disabled'] = nodes_from_both['communication'] - nodes_from_both['power']
        # modification
        Net = phase_3(Net,fraction_gen,fraction_power,nodes_from_both['disabled'])
        Net,G_power,G_com = phase_4(Net,G_power,G_com)
        return Net,G_power,G_com
    else:
        return Net,G_power,G_com





In [54]:
net = pn.case30()
G_com,Net,G_power,target,G_power_nodes_Number = create_net(net)
numer_of_nodes,of_set = Net.bus.shape[0], 0.2
p_frac = set_of_nodes(numer_of_nodes,of_set)
fraction_gen, fraction_power = 0.5,5
Net,G_power,G_com = casecading_version_2(Net,G_com,G_power,p_frac,fraction_gen,fraction_power)
print(Net.bus.shape[0])

1


In [55]:
20%10 == 0

True

In [58]:
k1 = []
for ii in range(1000):
    net = pn.case30()
    G_com,Net,G_power,target,G_power_nodes_Number = create_net(net)
    numer_of_nodes,of_set = Net.bus.shape[0], 0.2
    p_frac = set_of_nodes(numer_of_nodes,of_set)
    fraction_gen, fraction_power = 1,0.2
    Net,G_power,G_com = casecading_version_2(Net,G_com,G_power,p_frac,fraction_gen,fraction_power)
    k1.append(Net.bus.shape[0])
    if ii%100 == 0:
        print("oh we reached the {:.5}".format(ii/1000))

oh we reached the 0.0
oh we reached the 0.1
oh we reached the 0.2
oh we reached the 0.3
oh we reached the 0.4
oh we reached the 0.5
oh we reached the 0.6
oh we reached the 0.7
oh we reached the 0.8
oh we reached the 0.9


In [59]:
np.mean(k1)


16.09

In [34]:
np.mean(k1)

16.2458